Reduces the Full Catalog to a catalog containing only those SNe that have occurred within the overall PHANGS galaxies footprint

Saves catalog table as '../Data/1.PHANGS_SNe.csv'

Need to be on remote desktop to access files


Footprint taken from the sfsynthesis headers from Adam:
Big Galaxy Footprints
/data/tycho/0/leroy.42/projects/sfsynthesis/working_data/target_headers/<galaxy>_template<2or3d>_<restype>.fits

In [1]:
# IMPORT PACKAGES
import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table, vstack, unique
from astropy.wcs import WCS
from astropy.io import ascii
import astropy
from astropy.io import fits
import os.path
from os import path


In [2]:
# Import SN data

SN_Cat_File = '../Data/1.FullCatalog.csv'
SN_Cat = Table.read(SN_Cat_File, format='csv') 
SN_Cat[0]

# Import PHANGS Galaxy Names

PHANGS_File = '../Data/0.PHANGS_v1p6.csv'
PHANGS = Table.read(PHANGS_File, format='csv')
galaxies = PHANGS['galaxy'].astype(object) # removes the string truncating problem when editing galaxy names.
for i in range(len(galaxies)):   # Circinus is also known as ESO097-013
    if galaxies[i] == "circinus":
        galaxies[i] = "eso097-013"

# Construct FileName List

prefix = "/data/tycho/0/leroy.42/projects/sfsynthesis/working_data/target_headers/"
suffix = "_template2d_native.fits"
fileNames = [prefix + x + suffix for x in galaxies]
fileFlags = [True for x in galaxies]

for i in range(len(fileNames)):   #If no 3d file, use 2d
    
#     if path.isfile(fileNames[i]) == False:
#         fileNames[i] = fileNames[i].replace('3d', '2d')
        
    if path.isfile(fileNames[i]) == False: #If no 2d file, notify and remove this galaxy from sample
        print("No file for",galaxies[i])
        fileFlags[i] = False

print("There are", len(galaxies), "in the PHANGS Survey!")
print("we have footprint files for", len(np.where(fileFlags)[0]),"of them")

No file for eso097-013
No file for ic0010
No file for ic0342
No file for ic1954
No file for ic1993
No file for ic5273
No file for ic5332
No file for ngc0224
No file for ngc0247
No file for ngc0253
No file for ngc0278
No file for ngc0300
No file for ngc0598
No file for ngc0628
No file for ngc0685
No file for ngc0891
No file for ngc1068
No file for ngc1087
No file for ngc1097
No file for ngc1291
No file for ngc1313
No file for ngc1300
No file for ngc1317
No file for ngc1326
No file for ngc1365
No file for ngc1385
No file for ngc1433
No file for ngc1511
No file for ngc1512
No file for ngc1546
No file for ngc1559
No file for ngc1566
No file for ngc1637
No file for ngc1672
No file for ngc1809
No file for ngc1792
No file for ngc2090
No file for ngc2283
No file for ngc2403
No file for ngc2566
No file for ngc2683
No file for ngc2775
No file for ngc2835
No file for ngc2903
No file for ngc2997
No file for ngc3059
No file for ngc3031
No file for ngc3137
No file for ngc3184
No file for ngc3239
No 

In [3]:
# Function Calls

# function to see if a SN is in galaxy map
def check_in_image(SN_Cat, GalaxyImg): # ras and decs in decimal degrees
    
    hdulist = pyfits.open(GalaxyImg)
    map = hdulist[0].data
    wcs = WCS(hdulist[0].header, naxis=2)

    coords_arr = np.column_stack((SN_Cat["RA dds"], SN_Cat["Dec dds"])) # ras and decs now [ra,dec]
    pix_x, pix_y = wcs.wcs_world2pix(SN_Cat["RA dds"], SN_Cat["Dec dds"],0)

    #use world coordinates of all SNe to see if any fall in image (our version of footprint_contains)
    naxis = wcs._naxis #size of image
    naxis1 = hdulist[0].header['NAXIS1']
    naxis2 = hdulist[0].header['NAXIS2']
    is_in_x = (pix_x >= 0) & (pix_x <= naxis[0]-1) #because of 0-indexing
    is_in_y = (pix_y >= 0) & (pix_y <= naxis[1]-1)

    #get the name, ra, and dec of the SNe that fall in image
    #boolean array indexing (gives back array of Trues and Falses)
    #we are pulling out the SNe that are True and assigning them to own arrays
    name_in_image = np.array(SN_Cat["Name"])[is_in_x & is_in_y]
    type_in_image = np.array(SN_Cat["Type"])[is_in_x & is_in_y]
    ra_in_image = np.array(SN_Cat["RA dds"])[is_in_x & is_in_y]
    dec_in_image = np.array(SN_Cat["Dec dds"])[is_in_x & is_in_y]
    dd_in_image = np.array(SN_Cat["DiscDate"])[is_in_x & is_in_y]

    x_coord = np.array(pix_x)[is_in_x & is_in_y]
    y_coord = np.array(pix_y)[is_in_x & is_in_y]

    data=[]
    for k in range(len(name_in_image)):
        row = [name_in_image[k], type_in_image[k], ra_in_image[k], dec_in_image[k], dd_in_image[k]]
        data.append(row)
    return (data)

In [4]:
# Check in Footprint Images for SN

PHANGS_SNeData, PHANGS_Gal = [],[]

for i in range(len(fileNames)):
    if(fileFlags[i])==True:
        checkInImageData = check_in_image(SN_Cat, fileNames[i])
        for j in range(len(checkInImageData)):
            PHANGS_SNeData.append(checkInImageData[j])
            PHANGS_Gal.append(galaxies[i])
PHANGS_SNeData = np.asarray(PHANGS_SNeData)

In [6]:
PHANGS_SNeData

array([], dtype=float64)

In [5]:
## Compile SN info from table + galaxy name for PHANGS_SNe catalog

PHANGS_SNe = Table({"SNe Name": PHANGS_SNeData[:,0],
                    "SNe Type": PHANGS_SNeData[:,1],
                    "Galaxy"  : PHANGS_Gal,
                    "SNe Ra"  : PHANGS_SNeData[:,2],
                    "SNe Dec" : PHANGS_SNeData[:,3],
                    "Discovery Date": PHANGS_SNeData[:,4]})

PHANGS_SNe[0:2]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
# Write PHANGS_SNe catalog to file

ascii.write(PHANGS_SNe, '../Data/1.PHANGS_SNe.csv', format='csv', overwrite=True)
